This assignment focuses on developing a model to distinguish between AI-generated and human-written essays. To tackle this task, I began by importing all the necessary libraries, tools, and datasets for training and testing. I applied two different text preprocessing pipelines: one using stemming and the other using lemmatization, both of which included stopword removal and punctuation removal.

Next, I transformed the processed text data into numerical format using both CountVectorizer and TF-IDF, applying binary 1 and 0 (binary=True) and specifying a token pattern an ngram to capture words between one and three characters long.

I then trained separate machine learning models Support Vector Machines (SVM) and Decision Trees on each vectorized dataset. For SVM, I experimented with different kernels (linear and RBF), and for Decision Trees, I tested both the entropy and Gini criteria. I evaluated the performance of each model using various metric functions and compiled the results into a DataFrame using pandas.

Additionally, I plotted ROC curves for each model using scikit learn syntaxes, combining all SVM results into one graph and all Decision Tree results into another.

Finally, I used the Decision Tree model with the entropy criterion applied to the lemmatized data to predict the test set and generated a corresponding prediction file.


In [287]:
# Import necessary libraries for data manipulation, visualization, and machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import math
import time

# Sklearn libraries for machine learning and text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# NLTK libraries for text processing (lemmatization, stemming, stopwords, POS tagging)
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Set up visualization
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Download necessary NLTK resources for text processing
nltk.download('wordnet')  # WordNet for lemmatization
nltk.download('omw-1.4')  # Open Multilingual Wordnet
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords for text cleaning
nltk.download('averaged_perceptron_tagger')  # POS tagger for part-of-speech tagging
nltk.download('tagsets_json')  # Tagset resource

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Package tagsets_json is already up-to-date!


True

1.  Data loading and exploration

In [288]:
ds_train = pd.read_excel('AI_vs_huam_train_dataset.xlsx')
ds_test = pd.read_csv('Final_test_data.csv')

In [289]:
#Display data
ds_train.head()

essay  label
0  International sports events require the most w...      0
1  Globalisation has become a significant aspect ...      0
2  There is an ever-increasing number of bullying...      0
3  It is commonly believed, that companies should...      0
4  Despite knowing about the adverse effects of c...      0

In [290]:
ds_train['essay']

0       International sports events require the most w...
1       Globalisation has become a significant aspect ...
2       There is an ever-increasing number of bullying...
3       It is commonly believed, that companies should...
4       Despite knowing about the adverse effects of c...
                              ...                        
3723    I agree that successful people are willing to ...
3724    I believe that people who are open to trying n...
3725    To succeed in life, we must do everything poss...
3726    I'm not entirely sure what the word "successfu...
3727    The key to success is trying new things while ...
Name: essay, Length: 3728, dtype: object

In [291]:
ds_train.describe()

label
count  3728.000000
mean      0.500000
std       0.500067
min       0.000000
25%       0.000000
50%       0.500000
75%       1.000000
max       1.000000

2.  Text preprocessing functions:
remove stopword
lowercase
remove punc
stemming/limmatatization

processinig using stemming


In [292]:
def text_process_stemming(essay):

  # Handle non-string values
  if not isinstance(essay, str):
        return ""

  # Define a set of stopwords, including standard English stopwords and some custom ones
  STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']

  # Remove punctuation from the tweet
  nopunc = ''.join([char for char in essay if char not in string.punctuation])

  # Remove stopwords and join the words back into a string
  return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

  #Stemming
  stemming = PorterStemmer() #removes prefix and surfix using porterstemmer algorithm
  stemmed_tokens = [stemming.stem(token) for token in tokens]
  #Lemmatization
  #lemmatizer = WordNetLemmatizer()
  #return ' '.join([lemmatizer.lemmatize(word) for word in nopunc.split()])
  #lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

peprocessing using lemmatizer

In [293]:
def text_process_lemmatizer(essay):

  # Handle non-string values
  if not isinstance(essay, str):
        return ""

  # Define a set of stopwords, including standard English stopwords and some custom ones
  STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']

  # Remove punctuation from the tweet
  nopunc = ''.join([char for char in essay if char not in string.punctuation])

  # Remove stopwords and join the words back into a string
  return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

  #Lemmatization
  lemmatizer = WordNetLemmatizer()
  return ' '.join([lemmatizer.lemmatize(word) for word in nopunc.split()])
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

In [294]:
#Apply text cleaning to the "text" column
ds_train['stemmed_essay'] = ds_train['essay'].apply(text_process_stemming).copy()
ds_test['stemmed_essay'] = ds_test['essay'].apply(text_process_stemming) #train test datas so it is also familiar to those
ds_train.head()

essay  label  \
0  International sports events require the most w...      0   
1  Globalisation has become a significant aspect ...      0   
2  There is an ever-increasing number of bullying...      0   
3  It is commonly believed, that companies should...      0   
4  Despite knowing about the adverse effects of c...      0   

                                       stemmed_essay  
0  International sports events require welltraine...  
1  Globalisation become significant aspect world’...  
2  everincreasing number bullying activities nume...  
3  commonly believed companies dress code policy ...  
4  Despite knowing adverse effects climate change...

In [295]:
#Apply text cleaning to the "text" column
ds_train['lemmatized_essay'] = ds_train['essay'].apply(text_process_lemmatizer).copy()
ds_test['lemmatized_essay'] = ds_test['essay'].apply(text_process_lemmatizer) #train test datas so it is also familiar to those
ds_train.head()

essay  label  \
0  International sports events require the most w...      0   
1  Globalisation has become a significant aspect ...      0   
2  There is an ever-increasing number of bullying...      0   
3  It is commonly believed, that companies should...      0   
4  Despite knowing about the adverse effects of c...      0   

                                       stemmed_essay  \
0  International sports events require welltraine...   
1  Globalisation become significant aspect world’...   
2  everincreasing number bullying activities nume...   
3  commonly believed companies dress code policy ...   
4  Despite knowing adverse effects climate change...   

                                    lemmatized_essay  
0  International sports events require welltraine...  
1  Globalisation become significant aspect world’...  
2  everincreasing number bullying activities nume...  
3  commonly believed companies dress code policy ...  
4  Despite knowing adverse effects climate change...

How to compare vocabulary size before and after stop word removal?

In [296]:
# call the 2 vocabs
train_vocab_stem = set(" ".join(ds_train['stemmed_essay']).split())
train_vocab_lem = set(" ".join(ds_test['lemmatized_essay']).split())
non_train_vocab = set(" ".join(ds_train['essay']).split())


print("=== Vocabulary statistics ===")
print(f"Train vocab stemmed : {len(train_vocab_stem):,}")
print(f"Train vocab lemmatized: {len(train_vocab_stem):,}")
print(f"Untrained vocab : {len(non_train_vocab):,}")

=== Vocabulary statistics ===
Train vocab stemmed : 29,002
Train vocab lemmatized: 29,002
Untrained vocab : 42,702


3.  Feature extraction

Define x and y values for stemmed essays

In [297]:
#Define X and y
X = ds_train['stemmed_essay']
y = ds_train['label']
print("X: \n", X)
print("\ny: \n", y)

X: 
 0       International sports events require welltraine...
1       Globalisation become significant aspect world’...
2       everincreasing number bullying activities nume...
3       commonly believed companies dress code policy ...
4       Despite knowing adverse effects climate change...
                              ...                        
3723    agree successful people willing try new things...
3724    believe people open trying new things willing ...
3725    succeed life must everything possible‚Äîand so...
3726    entirely sure word successful really means use...
3727    key success trying new things minimizing risks...
Name: stemmed_essay, Length: 3728, dtype: object

y: 
 0       0
1       0
2       0
3       0
4       0
       ..
3723    1
3724    1
3725    1
3726    1
3727    1
Name: label, Length: 3728, dtype: int64


Define X and y values for lemmatized essays

In [298]:
X1 = ds_train['lemmatized_essay']
y1 = ds_train['label']
print("X: \n", X1)
print("\ny: \n", y1)

X: 
 0       International sports events require welltraine...
1       Globalisation become significant aspect world’...
2       everincreasing number bullying activities nume...
3       commonly believed companies dress code policy ...
4       Despite knowing adverse effects climate change...
                              ...                        
3723    agree successful people willing try new things...
3724    believe people open trying new things willing ...
3725    succeed life must everything possible‚Äîand so...
3726    entirely sure word successful really means use...
3727    key success trying new things minimizing risks...
Name: lemmatized_essay, Length: 3728, dtype: object

y: 
 0       0
1       0
2       0
3       0
4       0
       ..
3723    1
3724    1
3725    1
3726    1
3727    1
Name: label, Length: 3728, dtype: int64


Split Stemmed training data

In [299]:
#split training data into two parts (80% training dat and 20% test data)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Split Lemmatized traing data

In [300]:
#split training data into two parts (80% training dat and 20% test data)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, test_size=0.2, random_state=42, stratify=y)

Convert Stemmed preprocessed text into munerical feature using Countvectorizer

In [301]:
#Convert preprocessed text into munerical feature using Countvectorizer
Vectorizer = CountVectorizer(
    binary=True,
    token_pattern=r'\b\w+\b', #pattern (includes words with 1 char + more)
    ngram_range=(1, 1), #minimum 1 max words 1
    max_df=0.9,
    min_df=5,
    analyzer= "word"
    )

X_train_ds = Vectorizer.fit_transform(X_train)
X_val_ds = Vectorizer.transform(X_val)
print("\nVectorizer vocabulary size:", len(Vectorizer.get_feature_names_out()))
print("Sample features:", Vectorizer.get_feature_names_out())
print(X_val_ds.toarray())


Vectorizer vocabulary size: 5617
Sample features: ['1' '10' '100' ... 'äúsuccessful' 'äúthe' 'äúwhy']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Convert Lemmatized preprocessed text into munerical feature using Countvectorizer

In [302]:
#Convert preprocessed text into munerical feature using Countvectorizer
Vectorizer = CountVectorizer(
    binary=True,
    token_pattern=r'\b\w+\b', #pattern (includes words with 1 char + more)
    ngram_range=(1, 1), #minimum 1 max words 1
    max_df=0.9,
    min_df=5,
    analyzer= "word"
    )

X1_train_ds = Vectorizer.fit_transform(X1_train)
X1_val_ds = Vectorizer.transform(X1_val)
print("\nVectorizer vocabulary size:", len(Vectorizer.get_feature_names_out()))
print("Sample features:", Vectorizer.get_feature_names_out())
print(X1_val_ds.toarray())


Vectorizer vocabulary size: 5617
Sample features: ['1' '10' '100' ... 'äúsuccessful' 'äúthe' 'äúwhy']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Convert Stemmed preprocessed text into numerical feature using TF IDF

In [303]:
#Convert preprocessed text into numerical feature using TF IDF
Tfidf_Vectorizer = TfidfVectorizer(
    binary=True,
    token_pattern=r'\b\w+\b', #pattern (includes words with 1 char + more)
    ngram_range=(1, 1), #minimum 1 max words 1
    max_df=0.9,
    min_df=5
)

X_train_tfidf = Tfidf_Vectorizer.fit_transform(X_train)
X_val_tfidf = Tfidf_Vectorizer.transform(X_val)
print("\nVectorizer vocabulary size:", len(Tfidf_Vectorizer.get_feature_names_out()))
print("Sample features:", Tfidf_Vectorizer.get_feature_names_out())
print(X_val_tfidf.toarray())



Vectorizer vocabulary size: 5617
Sample features: ['1' '10' '100' ... 'äúsuccessful' 'äúthe' 'äúwhy']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Convert lemmatizsd preprocessed text into numerical feature using TF IDF

In [304]:
#Convert preprocessed text into numerical feature using TF IDF
Tfidf_Vectorizer = TfidfVectorizer(
    binary=True,
    token_pattern=r'\b\w+\b', #pattern (includes words with 1 char + more)
    ngram_range=(1, 1), #minimum 1 max words 1
    max_df=0.9,
    min_df=5
)

X1_train_tfidf = Tfidf_Vectorizer.fit_transform(X1_train)
X1_val_tfidf = Tfidf_Vectorizer.transform(X1_val)
print("\nVectorizer vocabulary size:", len(Tfidf_Vectorizer.get_feature_names_out()))
print("Sample features:", Tfidf_Vectorizer.get_feature_names_out())
print(X1_val_tfidf.toarray())


Vectorizer vocabulary size: 5617
Sample features: ['1' '10' '100' ... 'äúsuccessful' 'äúthe' 'äúwhy']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


4.  Model training and hyperparameter tuning

Implement SVM on stemmed countvectorized datas

svm using kernel linear

In [305]:
#Import Support Vector Machine
from sklearn import svm
start_time = time.time()
svm_linear_stm = svm.SVC(kernel='linear') #using parameter kernel = 'linear'
svm_linear_stm.fit(X_train_ds, y_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y_pred_class_SVMlinear = svm_linear_stm.predict(X_val_ds)


Training Time of SVM LINEAR: 2.588 seconds


svm using kernel rbf

In [306]:
#Import Support Vector Machine
from sklearn import svm
start_time = time.time()
svm_rbf_stm = svm.SVC(kernel='rbf') #using paramter kernel = 'rbf'
svm_rbf_stm.fit(X_train_ds, y_train)
end_time = time.time()
print("\nTraining Time of SVM rbf: {:.3f} seconds".format(end_time - start_time))
y_pred_class_SVMrbf = svm_rbf_stm.predict(X_val_ds)


Training Time of SVM rbf: 6.481 seconds


Implement SVM on stemmed TFIDFvectorized datas

svm using kernel linear

In [307]:
start_time = time.time()
svm_linear_stmtfidf = svm.SVC(kernel='linear') #using parameter kernel = 'linear'
svm_linear_stmtfidf.fit(X_train_tfidf, y_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y_pred_class_SVMlinear_tfidf = svm_linear_stmtfidf.predict(X_val_tfidf)


Training Time of SVM LINEAR: 4.701 seconds


svm using kernel rbf

In [308]:
start_time = time.time()
svm_rbf_stmtfidf = svm.SVC(kernel='rbf') #using parameter kernel = 'rbf'
svm_rbf_stmtfidf.fit(X_train_tfidf, y_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y_pred_class_SVMrbf_tfidf = svm_rbf_stmtfidf.predict(X_val_tfidf)


Training Time of SVM LINEAR: 10.169 seconds


Implement SVM on Lemmatized Countvectorized datas

svm using kernel linear

In [309]:
start_time = time.time()
svm_linear_lem = svm.SVC(kernel='linear') #using parameter kernel = 'linear'
svm_linear_lem.fit(X1_train_ds, y1_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y1_pred_class_SVMlinear = svm_linear_lem.predict(X1_val_ds)


Training Time of SVM LINEAR: 2.590 seconds


svm using kernel rbf

In [310]:
start_time = time.time()
svm_rbf_lem = svm.SVC(kernel='rbf') #using paramter kernel = 'rbf'
svm_rbf_lem.fit(X1_train_ds, y1_train)
end_time = time.time()
print("\nTraining Time of SVM rbf: {:.3f} seconds".format(end_time - start_time))
y1_pred_class_SVMrbf = svm_rbf_lem.predict(X1_val_ds)


Training Time of SVM rbf: 5.821 seconds


Implement SVM on Lemmatized TF-IDFVectorizer

svm using kernel linear

In [311]:
start_time = time.time()
svm_linear_lemtfidf = svm.SVC(kernel='linear') #using parameter kernel = 'linear'
svm_linear_lemtfidf.fit(X1_train_tfidf, y1_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y1_pred_class_SVMlinear_tfidf = svm_linear_lemtfidf.predict(X1_val_tfidf)


Training Time of SVM LINEAR: 4.741 seconds


svm using kernel rbf

In [312]:
start_time = time.time()
svm_rbf_lemtfidf = svm.SVC(kernel='rbf') #using parameter kernel = 'rbf'
svm_rbf_lemtfidf.fit(X1_train_tfidf, y1_train)
end_time = time.time()
print("\nTraining Time of SVM LINEAR: {:.3f} seconds".format(end_time - start_time))
y1_pred_class_SVMrbf_tfidf = svm_rbf_lemtfidf.predict(X1_val_tfidf)


Training Time of SVM LINEAR: 10.506 seconds


Implement Descision Tree(Dt)

Decision Tree on Stemmed CoutVectorized data

using entropy as criterion parameter

In [313]:
#Implement Decision Tree(DT)
from sklearn import tree #import tree from scikitlearn
start_time = time.time()
tree_entropy_stm = tree.DecisionTreeClassifier(
    criterion='entropy', #using 'entropy' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_entropy_stm.fit(X_train_ds, y_train)
end_time = time.time()
print("\nTraining Time of Decison Tree entropy stemmed data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y_pred_class_TREEentropy = tree_entropy_stm.predict(X_val_ds)


Training Time of Decison Tree entropy stemmed data: 0.328 seconds


using gini as criterion parameter

In [314]:
#Implement Decision Tree(DT)
from sklearn import tree #import tree from scikitlearn
start_time = time.time()
tree_gini_stm = tree.DecisionTreeClassifier(
    criterion='gini', #use 'gini' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_gini_stm.fit(X_train_ds, y_train)
end_time = time.time()
print("\nTraining Time of Decision Tree gini stemmed data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y_pred_class_TREEgini = tree_gini_stm.predict(X_val_ds)


Training Time of Decision Tree gini stemmed data: 0.216 seconds


Decision Tree on Stemmed TF-IDFVectorized data

using entropy as criterion parameter

In [315]:
start_time = time.time()
tree_entropy_stmtfidf = tree.DecisionTreeClassifier(
    criterion='entropy', #using 'entropy' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_entropy_stmtfidf.fit(X_train_tfidf, y_train)
end_time = time.time()
print("\nTraining Time of Decison Tree entropy tfidf stemmed data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y_pred_class_TREEentropy_tfidf = tree_entropy_stmtfidf.predict(X_val_tfidf)


Training Time of Decison Tree entropy tfidf stemmed data: 0.504 seconds


using gini as criterion parameter

In [316]:
start_time = time.time()
tree_gini_stmtfidf = tree.DecisionTreeClassifier(
    criterion='gini', #use 'gini' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_gini_stmtfidf.fit(X_train_tfidf, y_train)
end_time = time.time()
print("\nTraining Time of Decision Tree gini tfidf stemmed data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y_pred_class_TREEgini_tfidf = tree_gini_stmtfidf.predict(X_val_tfidf)


Training Time of Decision Tree gini tfidf stemmed data: 0.634 seconds


Decision Tree on Lemmatized CountVectorized data

using entropy as criterion parameter

In [317]:
#Implement Decision Tree(DT)
from sklearn import tree #import tree from scikitlearn
start_time = time.time()
tree_entropy_lem = tree.DecisionTreeClassifier(
    criterion='entropy', #using 'entropy' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_entropy_lem.fit(X1_train_ds, y1_train)
end_time = time.time()
print("\nTraining Time of Decison Tree entropy Lemmatized data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y1_pred_class_TREEentropy = tree_entropy_lem.predict(X1_val_ds)


Training Time of Decison Tree entropy Lemmatized data: 0.242 seconds


usinig gini as criterion parameter

In [318]:
start_time = time.time()
tree_gini_lem = tree.DecisionTreeClassifier(
    criterion='gini', #use 'gini' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_gini_lem.fit(X1_train_ds, y1_train)
end_time = time.time()
print("\nTraining Time of Decision Tree gini Lemmatized data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y1_pred_class_TREEgini = tree_gini_lem.predict(X1_val_ds)


Training Time of Decision Tree gini Lemmatized data: 0.232 seconds


Decision Tree on Lemmatized TF-IDFVectorized data

using entropy as criterion parameter

In [319]:
start_time = time.time()
tree_entropy_lemtfidf = tree.DecisionTreeClassifier(
    criterion='entropy', #using 'entropy' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_entropy_lemtfidf.fit(X1_train_tfidf, y1_train)
end_time = time.time()
print("\nTraining Time of Decison Tree entropy TFIDF Lemmatized data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y1_pred_class_TREEentropy_tfidf = tree_entropy_lemtfidf.predict(X1_val_tfidf)


Training Time of Decison Tree entropy TFIDF Lemmatized data: 0.541 seconds


using gini as criterion parameter

In [320]:
start_time = time.time()
tree_gini_lemtfidf = tree.DecisionTreeClassifier(
    criterion='gini', #use 'gini' as a criterion parameter
    max_depth= 10,
    min_samples_split= 5,
    min_samples_leaf= 5)
tree_gini_lemtfidf.fit(X1_train_tfidf, y1_train)
end_time = time.time()
print("\nTraining Time of Decision Tree gini tfidf Lemmatized data: {:.3f} seconds".format(end_time - start_time))
end_time = time.time()
y1_pred_class_TREEgini_tfidf = tree_gini_lemtfidf.predict(X1_val_tfidf)


Training Time of Decision Tree gini tfidf Lemmatized data: 0.658 seconds


5. Evaluation and visualization

Metrics for SVM

In [321]:
print("\n=======Metrics for SVM==========")
#Calculate and print the accuracy score
print("\n=======Accuracy Score===========")
# countvectorized stemmed svm linear accuracy
acc_SVMstemlinear = metrics.accuracy_score(y_val, y_pred_class_SVMlinear)
print(f"\nCountvectorized Stemmed linear Support Vector Machine Accuracy : {acc_SVMstemlinear*100:.2f}%")
#countvectorized stemmed svm rbf accuracy
acc_SVMstemrbf = metrics.accuracy_score(y_val, y_pred_class_SVMrbf)
print(f"\nCountvectorized Stemmed rbf Support Vector Machine Accuracy : {acc_SVMstemrbf*100:.2f}%")
#tfidfvectorized Stemmed linear
acc_SVMstemlinear_tfidf = metrics.accuracy_score(y_val, y_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Stemmed linear Support Vector Machine Accuracy : {acc_SVMstemlinear_tfidf*100:.2f}%")
#tfidfvectorized Stemmed rbf
acc_SVMstemrbf_tfidf = metrics.accuracy_score(y_val, y_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Stemmed rbf Support Vector Machine Accuracy : {acc_SVMstemrbf_tfidf*100:.2f}%")
#countvectorized Lemmatized svm linear
acc_SVMlemlinear = metrics.accuracy_score(y1_val, y1_pred_class_SVMlinear)
print(f"\nCountvectorized Lemmatized linear Support Vector Machine Accuracy : {acc_SVMlemlinear*100:.2f}%")
#countvectorized Lemmatized svm rbf
acc_SVMlemrbf = metrics.accuracy_score(y1_val, y1_pred_class_SVMrbf)
print(f"\nCountvectorized Lemmatized rbf Support Vector Machine Accuracy : {acc_SVMlemrbf*100:.2f}%")
#tfidfvectorized Lemmatized linear
acc_SVMlemlinear_tfidf = metrics.accuracy_score(y1_val, y1_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Lemmatized linear Support Vector Machine Accuracy : {acc_SVMlemlinear_tfidf*100:.2f}%")
#tfidfvectorized Lemmatized rbf
acc_SVMlemrbf_tfidf = metrics.accuracy_score(y1_val, y1_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Lemmatized rbf Support Vector Machine Accuracy : {acc_SVMlemrbf_tfidf*100:.2f}%")



# Print the precision score
print("\n=======Precision Score===========")
#Countervectorized stemmed svm linear precision
ps_SVMstemlinear = metrics.precision_score(y_val, y_pred_class_SVMlinear)
print(f"\nCountvectorized Stemmed linear Support Vector Machine Precision : {ps_SVMstemlinear*100:.2f}%")
#Countvectorized stemmed svm rbf precision
ps_SVMstemrbf = metrics.precision_score(y_val, y_pred_class_SVMrbf)
print(f"\nCountvectorized Stemmed rbf Support Vector Machine Precision : {ps_SVMstemrbf*100:.2f}%")
#tfidfvectorized stemmed svm linear
ps_SVMstemlinear_tfidf = metrics.precision_score(y_val, y_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Stemmed linear Support Vector Machine Precision : {ps_SVMstemlinear_tfidf*100:.2f}%")
#tfidfvectorized stemmed svm rbf
ps_SVMstemrbf_tfidf = metrics.precision_score(y_val, y_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Stemmed rbf Support Vector Machine Precision : {ps_SVMstemrbf_tfidf*100:.2f}%")
#countvectorized Lemmatized svm linear
ps_SVMlemlinear = metrics.precision_score(y1_val, y1_pred_class_SVMlinear)
print(f"\nCountvectorized Lemmatized linear Support Vector Machine Precision : {ps_SVMlemlinear*100:.2f}%")
#countvectorized Lemmatized svm rdf
ps_SVMlemrbf = metrics.precision_score(y1_val, y1_pred_class_SVMrbf)
print(f"\nCountvectorized Lemmatized rbf Support Vector Machine Precision : {ps_SVMlemrbf*100:.2f}%")
#tfidfvectorized Lemmatized linear
ps_SVMlemlinear_tfidf = metrics.precision_score(y1_val, y1_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Lemmatized linear Support Vector Machine Precision : {ps_SVMlemlinear_tfidf*100:.2f}%")
#tfidfvectorized Lemmatized rbf
ps_SVMlemrbf_tfidf = metrics.precision_score(y1_val, y1_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Lemmatized rbf Support Vector Machine Precision : {ps_SVMlemrbf_tfidf*100:.2f}%")


# Print the recall score
print("\n=======Recall Score===========")
#countvectorized stemmed svm linear precision
rs_SVMstemlinear = metrics.recall_score(y_val, y_pred_class_SVMlinear)
print(f"\nCountvectorized Stemmed linear Support Vector Machine Recall : {rs_SVMstemlinear*100:.2f}%")
#countvectorized stemmed svm brf
rs_SVMstemrbf = metrics.recall_score(y_val, y_pred_class_SVMrbf)
print(f"\nCountvectorized Stemmed rbf Support Vector Machine Recall : {rs_SVMstemrbf*100:.2f}%")
#tfidfvectorized stemmed svm linear
rs_SVMstemlinear_tfidf = metrics.recall_score(y_val, y_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Stemmed linear Support Vector Machine Recall : {rs_SVMstemlinear_tfidf*100:.2f}%")
#tfidfvectorized stemmed svm rbf
rs_SVMstemrbf_tfidf = metrics.recall_score(y_val, y_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Stemmed rbf Support Vector Machine Recall : {rs_SVMstemrbf_tfidf*100:.2f}%")
#countvertorized lemmatized svm linear
rs_SVMlemlinear = metrics.recall_score(y1_val, y1_pred_class_SVMlinear)
print(f"\nCountvectorized Lemmatized linear Support Vector Machine Recall : {rs_SVMlemlinear*100:.2f}%")
#countvectorized lemmatized svm rbf
rs_SVMlemrbf = metrics.recall_score(y1_val, y1_pred_class_SVMrbf)
print(f"\nCountvectorized Lemmatized rbf Support Vector Machine Recall : {rs_SVMlemrbf*100:.2f}%")
#tfidfvectorzed lemmarized svm linear
rs_SVMlemlinear_tfidf = metrics.recall_score(y1_val, y1_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Lemmatized linear Support Vector Machine Recall : {rs_SVMlemlinear_tfidf*100:.2f}%")
#tfidfvectorized lemmatized svm rbf
rs_SVMlemrbf_tfidf = metrics.recall_score(y1_val, y1_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Lemmatized rbf Support Vector Machine Recall : {rs_SVMlemrbf_tfidf*100:.2f}%")


#Print the F1-score
print("\n=======F1 Score===========")
#countvectorizer stemmed svm linear F1 score
f1_SVMstemlinear = metrics.f1_score(y_val, y_pred_class_SVMlinear)
print(f"\nCountvectorized Stemmed linear Support Vector Machine F1 Score : {f1_SVMstemlinear*100:.2f}%")
#countvectorized stemmed svm rbf
f1_SVMstemrbf = metrics.f1_score(y_val, y_pred_class_SVMrbf)
print(f"\nCountvectorized Stemmed rbf Support Vector Machine F1 Score : {f1_SVMstemrbf*100:.2f}%")
#tfidfvectorized stemmed svm linear
f1_SVMstemlinear_tfidf = metrics.f1_score(y_val, y_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Stemmed linear Support Vector Machine F1 Score : {f1_SVMstemlinear_tfidf*100:.2f}%")
#tfidfvectorized stemmed svm rbf
f1_SVMstemrbf_tfidf = metrics.f1_score(y_val, y_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Stemmed rbf Support Vector Machine F1 Score : {f1_SVMstemrbf_tfidf*100:.2f}%")
#countvectorized lemmatized svm linear
f1_SVMlemlinear = metrics.f1_score(y1_val, y1_pred_class_SVMlinear)
print(f"\nCountvectorized Lemmatized linear Support Vector Machine F1 Score : {f1_SVMlemlinear*100:.2f}%")
#countvectorized lemmatized svm rbf
f1_SVMlemrbf = metrics.f1_score(y1_val, y1_pred_class_SVMrbf)
print(f"\nCountvectorized Lemmatized rbf Support Vector Machine F1 Score : {f1_SVMlemrbf*100:.2f}%")
#tfidfvectorized lemmatized svm linear
f1_SVMlemlinear_tfidf = metrics.f1_score(y1_val, y1_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Lemmatized linear Support Vector Machine F1 Score : {f1_SVMlemlinear_tfidf*100:.2f}%")
#tfidfvectorized lemmatized svm rbf
f1_SVMlemrbf_tfidf = metrics.f1_score(y1_val, y1_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Lemmatized rbf Support Vector Machine F1 Score : {f1_SVMlemrbf_tfidf*100:.2f}%")


#Print the confusion score
print("\n=======Confusion Matrix===========")
#countvectorized stemmed svm linear
cm_SVMstemlinear = metrics.confusion_matrix(y_val, y_pred_class_SVMlinear)
print(f"\nCountvectorized Stemmed linear Support Vector Machine Confusion Matrix : \n{cm_SVMstemlinear}")
#countvectorized stemmed svm rbf
cm_SVMstemrbf = metrics.confusion_matrix(y_val, y_pred_class_SVMrbf)
print(f"\nCountvectorized Stemmed rbf Support Vector Machine Confusion Matrix : \n{cm_SVMstemrbf}")
#tfidfvectorized stemmed svm linear
cm_SVMstemlinear_tfidf = metrics.confusion_matrix(y_val, y_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Stemmed linear Support Vector Machine Confusion Matrix : \n{cm_SVMstemlinear_tfidf}")
#tfidfvectorized stemmed svm rbf
cm_SVMstemrbf_tfidf = metrics.confusion_matrix(y_val, y_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Stemmed rbf Support Vector Machine Confusion Matrix : \n{cm_SVMstemrbf_tfidf}")
#countvectorized lemmatized svm linear
cm_SVMlemlinear = metrics.confusion_matrix(y1_val, y1_pred_class_SVMlinear)
print(f"\nCountvectorized Lemmatized linear Support Vector Machine Confusion Matrix : \n{cm_SVMlemlinear}")
#countvectorized lemmatized svm rbf
cm_SVMlemrbf = metrics.confusion_matrix(y1_val, y1_pred_class_SVMrbf)
print(f"\nCountvectorized Lemmatized rbf Support Vector Machine Confusion Matrix : \n{cm_SVMlemrbf}")
#tfidfvectorized lemmatized svm linear
cm_SVMlemlinear_tfidf = metrics.confusion_matrix(y1_val, y1_pred_class_SVMlinear_tfidf)
print(f"\nTFIDFvectorized Lemmatized linear Support Vector Machine Confusion Matrix : \n{cm_SVMlemlinear_tfidf}")
#tfidfvectorized lemmatized svm rbf
cm_SVMlemrbf_tfidf = metrics.confusion_matrix(y1_val, y1_pred_class_SVMrbf_tfidf)
print(f"\nTFIDFvectorized Lemmatized rbf Support Vector Machine Confusion Matrix : \n{cm_SVMlemrbf_tfidf}")



=======Metrics for SVM==========

=======Accuracy Score===========

Countvectorized Stemmed linear Support Vector Machine Accuracy : 97.32%

Countvectorized Stemmed rbf Support Vector Machine Accuracy : 97.59%

TFIDFvectorized Stemmed linear Support Vector Machine Accuracy : 98.12%

TFIDFvectorized Stemmed rbf Support Vector Machine Accuracy : 97.99%

Countvectorized Lemmatized linear Support Vector Machine Accuracy : 97.32%

Countvectorized Lemmatized rbf Support Vector Machine Accuracy : 97.59%

TFIDFvectorized Lemmatized linear Support Vector Machine Accuracy : 98.12%

TFIDFvectorized Lemmatized rbf Support Vector Machine Accuracy : 97.99%

=======Precision Score===========

Countvectorized Stemmed linear Support Vector Machine Precision : 97.07%

Countvectorized Stemmed rbf Support Vector Machine Precision : 97.59%

TFIDFvectorized Stemmed linear Support Vector Machine Precision : 98.12%

TFIDFvectorized Stemmed rbf Support Vector Machine Precision : 98.38%

Countvectorized Lemmat

Metrix for Decision Tree

In [322]:
print("\n=======Metrics for SVM==========")
#Calculate and print the accuracy score
print("\n=======Accuracy Score===========")
# countvectorized stemmed TREE entropy accuracy
acc_TREEstementropy = metrics.accuracy_score(y_val, y_pred_class_TREEentropy)
print(f"\nCountvectorized Stemmed entropy Decision Tree Accuracy : {acc_TREEstementropy*100:.2f}%")
#countvectorized stemmed TREE gini accuracy
acc_TREEstemgini = metrics.accuracy_score(y_val, y_pred_class_TREEgini)
print(f"\nCountvectorized Stemmed gini Decision Tree Accuracy : {acc_TREEstemgini*100:.2f}%")
#tfidfvectorized Stemmed TREE entropy
acc_TREEstementropy_tfidf = metrics.accuracy_score(y_val, y_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Stemmed entropy Decision Tree Accuracy : {acc_TREEstementropy_tfidf*100:.2f}%")
#tfidfvectorized Stemmed TREE gini
acc_TREEstemgini_tfidf = metrics.accuracy_score(y_val, y_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Stemmed gini Decision Tree Accuracy : {acc_TREEstemgini_tfidf*100:.2f}%")
#countvectorized Lemmatized TREE entropy
acc_TREElementropy = metrics.accuracy_score(y1_val, y1_pred_class_TREEentropy)
print(f"\nCountvectorized Lemmatized entropy Decision Tree Accuracy : {acc_TREElementropy*100:.2f}%")
#countvectorized Lemmatized TREE gini
acc_TREElemgini = metrics.accuracy_score(y1_val, y1_pred_class_TREEgini)
print(f"\nCountvectorized Lemmatized gini Decision Tree Accuracy : {acc_TREElemgini*100:.2f}%")
#tfidfvectorized Lemmatized TREE entropy
acc_TREElementropy_tfidf = metrics.accuracy_score(y1_val, y1_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Lemmatized entropy Decision Tree Accuracy : {acc_TREElementropy_tfidf*100:.2f}%")
#tfidfvectorized Lemmatized tree gini
acc_TREElemgini_tfidf = metrics.accuracy_score(y1_val, y1_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Lemmatized gini Decision Tree Accuracy : {acc_TREElemgini_tfidf*100:.2f}%")



# Print the precision score
print("\n=======Precision Score===========")
#Countervectorized stemmed TREE entropy precision
ps_TREEstementropy = metrics.precision_score(y_val, y_pred_class_TREEentropy)
print(f"\nCountvectorized Stemmed entropy Decision Tree Precision : {ps_TREEstementropy*100:.2f}%")
#Countvectorized stemmed TREE gini
ps_TREEstemgini = metrics.precision_score(y_val, y_pred_class_TREEgini)
print(f"\nCountvectorized Stemmed gini Decision Tree Precision : {ps_TREEstemgini*100:.2f}%")
#tfidfvectorized Stemmed TREE entropy
ps_TREEstementropy_tfidf = metrics.precision_score(y_val, y_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Stemmed entropy Decision Tree Precision : {ps_TREEstementropy_tfidf*100:.2f}%")
#tfidfvectorized stemmed TREE gini
ps_TREEstemgini_tfidf = metrics.precision_score(y_val, y_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Stemmed gini Decision Tree Precision : {ps_TREEstemgini_tfidf*100:.2f}%")
#countvectorized Lemmatized TREE entropy
ps_TREElementropy = metrics.precision_score(y1_val, y1_pred_class_TREEentropy)
print(f"\nCountvectorized Lemmatized entropy Decision Tree Precision : {ps_TREElementropy*100:.2f}%")
#countvectorized Lemmatized TREE gini
ps_TREElemgini = metrics.precision_score(y1_val, y1_pred_class_TREEgini)
print(f"\nCountvectorized Lemmatized gini Decision Tree Precision : {ps_TREElemgini*100:.2f}%")
#tfidfvectorized Lemmatized TREE entropy
ps_TREElementropy_tfidf = metrics.precision_score(y1_val, y1_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Lemmatized entropy Decision Tree Precision : {ps_TREElementropy_tfidf*100:.2f}%")
#tfidfvectorized Lemmatized TREE gini
ps_TREElemgini_tfidf = metrics.precision_score(y1_val, y1_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Lemmatized gini Decision Tree Precision : {ps_TREElemgini_tfidf*100:.2f}%")


# Print the recall score
print("\n=======Recall Score===========")
#countvectorized stemmed TREE entropy
rs_TREEstementropy = metrics.recall_score(y_val, y_pred_class_TREEentropy)
print(f"\nCountvectorized Stemmed entropy Decision Tree Recall : {rs_TREEstementropy*100:.2f}%")
#countvectorized stemmed TREE gini
rs_TREEstemgini = metrics.recall_score(y_val, y_pred_class_TREEgini)
print(f"\nCountvectorized Stemmed gini Decision Tree Recall : {rs_TREEstemgini*100:.2f}%")
#tfidfvectorized stemmed TREE entropy
rs_TREEstementropy_tfidf = metrics.recall_score(y_val, y_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Stemmed entropy Decision Tree Recall : {rs_TREEstementropy_tfidf*100:.2f}%")
#tfidfvectorized stemmed TREE gini
rs_TREEstemgini_tfidf = metrics.recall_score(y_val, y_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Stemmed gini Decision Tree Recall : {rs_TREEstemgini_tfidf*100:.2f}%")
#countvectorized lemmatized TREE entropy
rs_TREElementropy = metrics.recall_score(y1_val, y1_pred_class_TREEentropy)
print(f"\nCountvectorized Lemmatized entropy Decision Tree Recall : {rs_TREElementropy*100:.2f}%")
#countvectorized lemmatized TREE gini
rs_TREElemgini = metrics.recall_score(y1_val, y1_pred_class_TREEgini)
print(f"\nCountvectorized Lemmatized gini Decision Tree Recall : {rs_TREElemgini*100:.2f}%")
#tfidfvectorized lemmatized TREE entropy
rs_TREElementropy_tfidf = metrics.recall_score(y1_val, y1_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Lemmatized entropy Decision Tree Recall : {rs_TREElementropy_tfidf*100:.2f}%")
#tfidfvectorized lemmatized TREE gini
rs_TREElemgini_tfidf = metrics.recall_score(y1_val, y1_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Lemmatized gini Decision Tree Recall : {rs_TREElemgini_tfidf*100:.2f}%")

#Print the F1-score
print("\n=======F1 Score===========")
#countvectorizer stemmed TREE entropy F1 score
f1_TREEstementropy = metrics.f1_score(y_val, y_pred_class_TREEentropy)
print(f"\nCountvectorized Stemmed entropy Decision Tree F1 Score : {f1_TREEstementropy*100:.2f}%")
#countvectorizer stemmed TREE gini F1 score
f1_TREEstemgini = metrics.f1_score(y_val, y_pred_class_TREEgini)
print(f"\nCountvectorized Stemmed gini Decision Tree F1 Score : {f1_TREEstemgini*100:.2f}%")
#tfidfvectorized stemmed TREE entropy
f1_TREEstementropy_tfidf = metrics.f1_score(y_val, y_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Stemmed entropy Decision Tree F1 Score : {f1_TREEstementropy_tfidf*100:.2f}%")
#tfidfvectorized stemmed TREE gini
f1_TREEstemgini_tfidf = metrics.f1_score(y_val, y_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Stemmed gini Decision Tree F1 Score : {f1_TREEstemgini_tfidf*100:.2f}%")
#countvectorizer lemmatized TREE entropy
f1_TREElementropy = metrics.f1_score(y1_val, y1_pred_class_TREEentropy)
print(f"\nCountvectorized Lemmatized entropy Decision Tree F1 Score : {f1_TREElementropy*100:.2f}%")
#countvectorizer lemmatized TREE gini
f1_TREElemgini = metrics.f1_score(y1_val, y1_pred_class_TREEgini)
print(f"\nCountvectorized Lemmatized gini Decision Tree F1 Score : {f1_TREElemgini*100:.2f}%")
#tfidfvectorized lemmatized TREE entropy
f1_TREElementropy_tfidf = metrics.f1_score(y1_val, y1_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Lemmatized entropy Decision Tree F1 Score : {f1_TREElementropy_tfidf*100:.2f}%")
#tfidfvectorized lemmatized TREE gini
f1_TREElemgini_tfidf = metrics.f1_score(y1_val, y1_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Lemmatized gini Decision Tree F1 Score : {f1_TREElemgini_tfidf*100:.2f}%")

#Print the confusion score
print("\n=======Confusion Matrix===========")
#countvectorized stemmed TREE entropy
cm_TREEstementropy = metrics.confusion_matrix(y_val, y_pred_class_TREEentropy)
print(f"\nCountvectorized Stemmed entropy Decision Tree Confusion Matrix : \n{cm_TREEstementropy}")
#countvectorizer stemmed TREE gini
cm_TREEstemgini = metrics.confusion_matrix(y_val, y_pred_class_TREEgini)
print(f"\nCountvectorized Stemmed gini Decision Tree Confusion Matrix : \n{cm_TREEstemgini}")
#tfidfvectorized stemmed TREE entropy
cm_TREEstementropy_tfidf = metrics.confusion_matrix(y_val, y_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Stemmed entropy Decision Tree Confusion Matrix : \n{cm_TREEstementropy_tfidf}")
#tfidfvectorized stemmed TREE gini
cm_TREEstemgini_tfidf = metrics.confusion_matrix(y_val, y_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Stemmed gini Decision Tree Confusion Matrix : \n{cm_TREEstemgini_tfidf}")
#countvectorizer lemmatized TREE entropy
cm_TREElementropy = metrics.confusion_matrix(y1_val, y1_pred_class_TREEentropy)
print(f"\nCountvectorized Lemmatized entropy Decision Tree Confusion Matrix : \n{cm_TREElementropy}")
#countvectorizer lemmatized TREE gini
cm_TREElemgini = metrics.confusion_matrix(y1_val, y1_pred_class_TREEgini)
print(f"\nCountvectorized Lemmatized gini Decision Tree Confusion Matrix : \n{cm_TREElemgini}")
#tfidfvectorized lemmatized TREE entropy
cm_TREElementropy_tfidf = metrics.confusion_matrix(y1_val, y1_pred_class_TREEentropy_tfidf)
print(f"\nTFIDFvectorized Lemmatized entropy Decision Tree Confusion Matrix : \n{cm_TREElementropy_tfidf}")
#tfidfvectorized lemmatized TREE gini
cm_TREElemgini_tfidf = metrics.confusion_matrix(y1_val, y1_pred_class_TREEgini_tfidf)
print(f"\nTFIDFvectorized Lemmatized gini Decision Tree Confusion Matrix : \n{cm_TREElemgini_tfidf}")



=======Metrics for SVM==========

=======Accuracy Score===========

Countvectorized Stemmed entropy Decision Tree Accuracy : 91.02%

Countvectorized Stemmed gini Decision Tree Accuracy : 92.36%

TFIDFvectorized Stemmed entropy Decision Tree Accuracy : 90.35%

TFIDFvectorized Stemmed gini Decision Tree Accuracy : 91.55%

Countvectorized Lemmatized entropy Decision Tree Accuracy : 90.75%

Countvectorized Lemmatized gini Decision Tree Accuracy : 92.23%

TFIDFvectorized Lemmatized entropy Decision Tree Accuracy : 90.48%

TFIDFvectorized Lemmatized gini Decision Tree Accuracy : 91.29%

=======Precision Score===========

Countvectorized Stemmed entropy Decision Tree Precision : 92.27%

Countvectorized Stemmed gini Decision Tree Precision : 92.93%

TFIDFvectorized Stemmed entropy Decision Tree Precision : 90.35%

TFIDFvectorized Stemmed gini Decision Tree Precision : 91.22%

Countvectorized Lemmatized entropy Decision Tree Precision : 92.94%

Countvectorized Lemmatized gini Decision Tree Pre

In [323]:
# Create a dictionary to store the performance metrics for each model
performance_data = {
    'Model': [],
    'Vectorizer': [],
    'Preprocessing': [],
    'Kernel/Criterion': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': []
}

# Add SVM metrics to the dictionary
performance_data['Model'].extend(['SVM'] * 8)
performance_data['Vectorizer'].extend(['CountVectorizer', 'CountVectorizer', 'TF-IDF', 'TF-IDF'] * 2)
performance_data['Preprocessing'].extend(['Stemmed'] * 4 + ['Lemmatized'] * 4)
performance_data['Kernel/Criterion'].extend(['Linear', 'RBF'] * 4)
performance_data['Accuracy'].extend([
    acc_SVMstemlinear, acc_SVMstemrbf, acc_SVMstemlinear_tfidf, acc_SVMstemrbf_tfidf,
    acc_SVMlemlinear, acc_SVMlemrbf, acc_SVMlemlinear_tfidf, acc_SVMlemrbf_tfidf
])
performance_data['Precision'].extend([
    ps_SVMstemlinear, ps_SVMstemrbf, ps_SVMstemlinear_tfidf, ps_SVMstemrbf_tfidf,
    ps_SVMlemlinear, ps_SVMlemrbf, ps_SVMlemlinear_tfidf, ps_SVMlemrbf_tfidf
])
performance_data['Recall'].extend([
    rs_SVMstemlinear, rs_SVMstemrbf, rs_SVMstemlinear_tfidf, rs_SVMstemrbf_tfidf,
    rs_SVMlemlinear, rs_SVMlemrbf, rs_SVMlemlinear_tfidf, rs_SVMlemrbf_tfidf
])
performance_data['F1 Score'].extend([
    f1_SVMstemlinear, f1_SVMstemrbf, f1_SVMstemlinear_tfidf, f1_SVMstemrbf_tfidf,
    f1_SVMlemlinear, f1_SVMlemrbf, f1_SVMlemlinear_tfidf, f1_SVMlemrbf_tfidf
])

# Add Decision Tree metrics to the dictionary
performance_data['Model'].extend(['Decision Tree'] * 8)
performance_data['Vectorizer'].extend(['CountVectorizer', 'CountVectorizer', 'TF-IDF', 'TF-IDF'] * 2)
performance_data['Preprocessing'].extend(['Stemmed'] * 4 + ['Lemmatized'] * 4)
performance_data['Kernel/Criterion'].extend(['Entropy', 'Gini'] * 4)
performance_data['Accuracy'].extend([
    acc_TREEstementropy, acc_TREEstemgini, acc_TREEstementropy_tfidf, acc_TREEstemgini_tfidf,
    acc_TREElementropy, acc_TREElemgini, acc_TREElementropy_tfidf, acc_TREElemgini_tfidf
])
performance_data['Precision'].extend([
    ps_TREEstementropy, ps_TREEstemgini, ps_TREEstementropy_tfidf, ps_TREEstemgini_tfidf,
    ps_TREElementropy, ps_TREElemgini, ps_TREElementropy_tfidf, ps_TREElemgini_tfidf
])
performance_data['Recall'].extend([
    rs_TREEstementropy, rs_TREEstemgini, rs_TREEstementropy_tfidf, rs_TREEstemgini_tfidf,
    rs_TREElementropy, rs_TREElemgini, rs_TREElementropy_tfidf, rs_TREElemgini_tfidf
])
performance_data['F1 Score'].extend([
    f1_TREEstementropy, f1_TREEstemgini, f1_TREEstementropy_tfidf, f1_TREEstemgini_tfidf,
    f1_TREElementropy, f1_TREElemgini, f1_TREElementropy_tfidf, f1_TREElemgini_tfidf
])

# Create the performance table using pandas DataFrame
performance_table = pd.DataFrame(performance_data)

# Format the numerical columns to percentages
performance_table['Accuracy'] = performance_table['Accuracy'].apply(lambda x: f'{x:.2%}')
performance_table['Precision'] = performance_table['Precision'].apply(lambda x: f'{x:.2%}')
performance_table['Recall'] = performance_table['Recall'].apply(lambda x: f'{x:.2%}')
performance_table['F1 Score'] = performance_table['F1 Score'].apply(lambda x: f'{x:.2%}')

# Display the table
print("\n======= Performance Comparison Table (SVM vs Decision Tree) =======\n")
print(performance_table.to_string())



======= Performance Comparison Table (SVM vs Decision Tree) =======

            Model       Vectorizer Preprocessing Kernel/Criterion Accuracy Precision  Recall F1 Score
0             SVM  CountVectorizer       Stemmed           Linear   97.32%    97.07%  97.59%   97.33%
1             SVM  CountVectorizer       Stemmed              RBF   97.59%    97.59%  97.59%   97.59%
2             SVM           TF-IDF       Stemmed           Linear   98.12%    98.12%  98.12%   98.12%
3             SVM           TF-IDF       Stemmed              RBF   97.99%    98.38%  97.59%   97.98%
4             SVM  CountVectorizer    Lemmatized           Linear   97.32%    97.07%  97.59%   97.33%
5             SVM  CountVectorizer    Lemmatized              RBF   97.59%    97.59%  97.59%   97.59%
6             SVM           TF-IDF    Lemmatized           Linear   98.12%    98.12%  98.12%   98.12%
7             SVM           TF-IDF    Lemmatized              RBF   97.99%    98.38%  97.59%   97.98%
8   Decision

In [328]:
from sklearn.metrics import roc_curve, auc

# plot the ROC for the SVM linear model trained on stemmed CountVectorized data
# Get the decision function scores
y_scores_svm_linear_stm = svm_linear_stm.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr1, tpr1, thresholds = roc_curve(y_val, y_scores_svm_linear_stm)
# Calculate the AUC (Area Under the Curve)
roc_auc1 = auc(fpr1, tpr1)

# plot the ROC for the SVM rbf model trained on stemmed CountVectorized data
# Get the decision function scores
y_scores_svm_rbf_stm = svm_rbf_stm.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr2, tpr2, thresholds = roc_curve(y_val, y_scores_svm_rbf_stm)
# Calculate the AUC (Area Under the Curve)
roc_auc2 = auc(fpr2, tpr2)

# plot the ROC for the SVM linear model trained on stemmed tfidfVectorized data
# Get the decision function scores
y_scores_svm_linear_stmtfidf = svm_linear_stmtfidf.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr3, tpr3, thresholds = roc_curve(y_val, y_scores_svm_linear_stmtfidf)
# Calculate the AUC (Area Under the Curve)
roc_auc3 = auc(fpr3, tpr3)

# plot the ROC for the SVM rbf model trained on stemmed tfidfVectorized data
# Get the decision function scores
y_scores_svm_rbf_stmtfidf = svm_rbf_stmtfidf.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr4, tpr4, thresholds = roc_curve(y_val, y_scores_svm_rbf_stmtfidf)
# Calculate the AUC (Area Under the Curve)
roc_auc4 = auc(fpr4, tpr4)

# plot the ROC for the SVM linear model trained on lemmatized CountVectorized data
# Get the decision function scores
y_scores_svm_linear_lem = svm_linear_lem.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr5, tpr5, thresholds = roc_curve(y_val, y_scores_svm_linear_lem)
# Calculate the AUC (Area Under the Curve)
roc_auc5 = auc(fpr5, tpr5)

# plot the ROC for the SVM rbf model trained on lemmatized CountVectorized data
# Get the decision function scores
y_scores_svm_rbf_lem = svm_rbf_lem.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr6, tpr6, thresholds = roc_curve(y_val, y_scores_svm_rbf_lem)
# Calculate the AUC (Area Under the Curve)
roc_auc6 = auc(fpr6, tpr6)

# plot the ROC for the SVM linear model trained on lemmatized tfidfVectorized data
# Get the decision function scores
y_scores_svm_linear_lemtfidf = svm_linear_lemtfidf.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr7, tpr7, thresholds = roc_curve(y_val, y_scores_svm_linear_lemtfidf)
# Calculate the AUC (Area Under the Curve)
roc_auc7 = auc(fpr7, tpr7)

# plot the ROC for the SVM rbf model trained on lemmatized tfidfVectorized data
# Get the decision function scores
y_scores_svm_rbf_lemtfidf = svm_rbf_lemtfidf.decision_function(X_val_ds)
# Calculate the ROC curve points
fpr8, tpr8, thresholds = roc_curve(y_val, y_scores_svm_rbf_lemtfidf)
# Calculate the AUC (Area Under the Curve)
roc_auc8 = auc(fpr8, tpr8)

print("ROC CURVE FOR SVM")
# Plot the ROC curve
plt.figure()
plt.plot(fpr1, tpr1, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr2, tpr2, color='red', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr3, tpr3, color='yellow', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr4, tpr4, color='green', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr5, tpr5, color='black', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr6, tpr6, color='brown', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr7, tpr7, color='purple', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot(fpr8, tpr8, color='pink', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve SVM')
plt.legend(loc="lower right")
plt.show()


ROC CURVE FOR SVM


In [329]:

# plot the ROC for the TREE entropy model trained on stemmed CountVectorized data
y_scores_tree_entropy_stm = tree_entropy_stm.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree1, tpr_tree1, thresholds_tree = roc_curve(y_val, y_scores_tree_entropy_stm)
roc_auc_tree = auc(fpr_tree1, tpr_tree1)

# plot the ROC for the TREE gini model trained on stemmed CountVectorized data
y_scores_tree_gini_stm = tree_gini_stm.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree2, tpr_tree2, thresholds_tree = roc_curve(y_val, y_scores_tree_gini_stm)
roc_auc_tree = auc(fpr_tree2, tpr_tree2)

# plot the ROC for the TREE entropy model trained on stemmed tfidfVectorized data
y_scores_tree_entropy_stmtfidf = tree_entropy_stmtfidf.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree3, tpr_tree3, thresholds_tree = roc_curve(y_val, y_scores_tree_entropy_stmtfidf)
roc_auc_tree = auc(fpr_tree3, tpr_tree3)

# plot the ROC for the TREE gini model trained on stemmed tfidfVectorized data
y_scores_tree_gini_stmtfidf = tree_gini_stmtfidf.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree4, tpr_tree4, thresholds_tree = roc_curve(y_val, y_scores_tree_gini_stmtfidf)
roc_auc_tree = auc(fpr_tree4, tpr_tree4)

# plot the ROC for the TREE entropy model trained on lemmatized CountVectorized data
y_scores_tree_entropy_lem = tree_entropy_lem.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree5, tpr_tree5, thresholds_tree = roc_curve(y_val, y_scores_tree_entropy_lem)
roc_auc_tree = auc(fpr_tree5, tpr_tree5)

# plot the ROC for the TREE gini model trained on lemmatized CountVectorized data
y_scores_tree_gini_lem = tree_gini_lem.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree6, tpr_tree6, thresholds_tree = roc_curve(y_val, y_scores_tree_gini_lem)
roc_auc_tree = auc(fpr_tree6, tpr_tree6)

# plot the ROC for the TREE entropy model trained on lemmatized tfidfVectorized data
y_scores_tree_entropy_lemtfidf = tree_entropy_lemtfidf.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree7, tpr_tree7, thresholds_tree = roc_curve(y_val, y_scores_tree_entropy_lemtfidf)
roc_auc_tree = auc(fpr_tree7, tpr_tree7)

# plot the ROC for the TREE gini model trained on lemmatized tfidfVectorized data
y_scores_tree_gini_lemtfidf = tree_gini_lemtfidf.predict_proba(X_val_ds)[:, 1] # Get probability of the positive class
fpr_tree8, tpr_tree8, thresholds_tree = roc_curve(y_val, y_scores_tree_gini_lemtfidf)
roc_auc_tree = auc(fpr_tree8, tpr_tree8)

# To plot multiple ROC curves on the same plot:
plt.figure()
plt.plot(fpr_tree1, tpr_tree1, color='blue', lw=2, label=f'Decision Tree Entropy Stemmed (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree2, tpr_tree2, color='red', lw=2, label=f'Decision Tree Gini Stemmed (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree3, tpr_tree3, color='green', lw=2, label=f'Decision Tree Entropy Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree4, tpr_tree4, color='yellow', lw=2, label=f'Decision Tree Gini Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree5, tpr_tree5, color='purple', lw=2, label=f'Decision Tree Entropy Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree6, tpr_tree6, color='orange', lw=2, label=f'Decision Tree Gini Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree7, tpr_tree7, color='brown', lw=2, label=f'Decision Tree Entropy Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot(fpr_tree8, tpr_tree8, color='pink', lw=2, label=f'Decision Tree Gini Lemmatized (area = {roc_auc_tree:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves DT')
plt.legend(loc="lower right")
plt.show()

In [336]:
ds_test

essay  label  \
0    Enjoyment means entertainment, satisfaction of...    NaN   
1    Before to start with my opinion of the topic i...    NaN   
2    Thats a really really good subject, but acctul...    NaN   
3    I disagree with the statement saying that it i...    NaN   
4    It is undeniable that advertisements play a si...    NaN   
..                                                 ...    ...   
864  Advertisements occupy a big part in our lives ...    NaN   
865  Young people are the fundamental building bloc...    NaN   
866  In my opinion, and experience, old people can ...    NaN   
867  I agree with this statement that most advertis...    NaN   
868  It is undeniable that advertisements play a si...    NaN   

                                         stemmed_essay  \
0    Enjoyment means entertainment satisfaction one...   
1    start opinion topic must say life much importa...   
2    Thats really really good subject acctully say ...   
3    disagree statement saying better broad knowled...   
4    undeniable advertisements play significant rol...   
..                                                 ...   
864  Advertisements occupy big part lives every com...   
865  Young people fundamental building blocks commu...   
866  opinion experience old people enjoy life much ...   
867  agree statement advertisements make products s...   
868  undeniable advertisements play significant rol...   

                                      lemmatized_essay  
0    Enjoyment means entertainment satisfaction one...  
1    start opinion topic must say life much importa...  
2    Thats really really good subject acctully say ...  
3    disagree statement saying better broad knowled...  
4    undeniable advertisements play significant rol...  
..                                                 ...  
864  Advertisements occupy big part lives every com...  
865  Young people fundamental building blocks commu...  
866  opinion experience old people enjoy life much ...  
867  agree statement advertisements make products s...  
868  undeniable advertisements play significant rol...  

[869 rows x 4 columns]

Convert lemmatized essay from test using tdidf vectorizer and test using test. And create a prediction file


In [339]:
X_test = Tfidf_Vectorizer.transform(ds_test['lemmatized_essay'])
label_pred_class_TREEentropy_tfidf = tree_entropy_lemtfidf.predict(X_test)
label_pred_class_TREEentropy_tfidf


array([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

In [357]:
# Create the output DataFrame
Prediction_df = pd.DataFrame({'essay': ds_test['essay'], 'label': label_pred_class_TREEentropy_tfidf})

# Display the predicted labels
print("\nPredicted labels for test data:")
Prediction_df


Predicted labels for test data:


essay  label
0    Enjoyment means entertainment, satisfaction of...      0
1    Before to start with my opinion of the topic i...      0
2    Thats a really really good subject, but acctul...      0
3    I disagree with the statement saying that it i...      1
4    It is undeniable that advertisements play a si...      0
..                                                 ...    ...
864  Advertisements occupy a big part in our lives ...      0
865  Young people are the fundamental building bloc...      0
866  In my opinion, and experience, old people can ...      0
867  I agree with this statement that most advertis...      0
868  It is undeniable that advertisements play a si...      0

[869 rows x 2 columns]

In [351]:
#Create the prediction file
Prediction_df.to_csv('Prediction.csv', index=False)